In [1]:
#event
def process_data(data,callback):
    print('Process data ...')
    processed_data = data + " is processed."
    callback(processed_data)

#callback
def print_result(result):
    print('reult: ', result)


process_data('sample data',print_result)

Process data ...
reult:  sample data is processed.


## test with keypad

In [ ]:
import time
from pymata4 import pymata4

class Keypad:
    rows = [2, 3, 4, 5]
    cols = [6, 7, 8, 9]
    keypad_layout = [
        ['1', '2', '3', 'A'],
        ['4', '5', '6', 'B'],
        ['7', '8', '9', 'C'],
        ['*', '0', '#', 'D']
    ]

    def _init_(self, board, callback=None):
        self.board = board
        self.callback = callback
        self.current_row = 0  # Start scanning with the first row
        self.setup_pins()
        self.activate_row()
        self.entered_code = []
        self.last_key_time = time.time()
        self.correct_code = ['3', '3', '9']
        self.finish_code = ['D', 'D', 'D']

    def setup_pins(self):
        # Set row pins as outputs and columns as inputs with pull-up
        for row in self.rows:
            self.board.set_pin_mode_digital_output(row)
            self.board.digital_write(row, 1)  # Set all rows high (inactive)
        for col in self.cols:
            self.board.set_pin_mode_digital_input_pullup(col, self.pin_change_callback)

    def activate_row(self):
        # Deactivate all rows
        for row in self.rows:
            self.board.digital_write(row, 1)
        # Activate the current row
        self.board.digital_write(self.rows[self.current_row], 0)

    def pin_change_callback(self, data):
        pin, value = data[1], data[2]
        if value == 0:  # If state is low, a button is pressed
            key = self.keypad_layout[self.current_row][self.cols.index(pin)]
            self.toy_example(key)

    def toy_example(self, key):
        current_time = time.time()
        if current_time - self.last_key_time > 5:
            self.entered_code = []  # Reset if too long between presses
            print("Resetting due to timeout...")
        self.last_key_time = current_time
        print(f"Key Pressed: {key}")
        self.entered_code.append(key)

        if len(self.entered_code) == 3:
            if self.entered_code == self.correct_code:
                print("Code Correct. Access Granted.")
                for _ in range(5):  # Display access signal
                    print(".", end=" ", flush=True)
                    time.sleep(1)
                print("\nResetting for new code...")
                self.entered_code = []
            elif self.entered_code == self.finish_code:
                print("Stop")
                self.stop = True  # Set a flag to stop the loop
            else:
                print("Incorrect Code. Try again.")
                self.entered_code = []

    def start(self):
        self.stop = False  # Initialize stop flag
        print("Starting digicode system. Please wait...")
        time.sleep(1)
        print("Ready for code entry...")
        try:
            while not self.stop:
                self.activate_row()
                time.sleep(0.1)  # Shorter delay to improve responsiveness
                self.current_row = (self.current_row + 1) % len(self.rows)  # Cycle through the rows
        except KeyboardInterrupt:
            print("Program terminated by user.")
        finally:
            self.board.shutdown()  # Ensure clean shutdown
            print("System shutdown cleanly.")


if __name__ == "_main_":
    board = pymata4.Pymata4()
    keypad = Keypad(board)
    keypad.start()